In [1]:
# Python library imports: numpy, random, sklearn, pandas, etc

import warnings
warnings.filterwarnings('ignore')

import sys
import random
import numpy as np

# cross_validation is deprecated since version 0.18. This module will be removed in 0.20.
# Use sklearn.model_selection.train_test_split instead.
# from sklearn import linear_model, cross_validation, metrics, svm
from sklearn.model_selection import train_test_split
from sklearn import linear_model, metrics, svm
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# function to read HDFS file into dataframe using PyDoop
import pydoop.hdfs as hdfs
def read_csv_from_hdfs(path, cols, col_types=None):
    files = hdfs.ls(path);
    pieces = []
    for f in files:
        fhandle = hdfs.open(f)
        pieces.append(pd.read_csv(fhandle, names=cols, dtype=col_types, skiprows=1))
        fhandle.close()
    return pd.concat(pieces, ignore_index=True)

In [2]:
# this array defines the dates of holiday in 2007 and 2008 
# holidays = [ date(2007, 1, 1), date(2007, 1, 15), date(2007, 2, 19), date(2007, 5, 28), date(2007, 6, 7), date(2007, 7, 4),  date(2007, 9, 3), date(2007, 10, 8), date(2007, 11, 11), date(2007, 11, 22), date(2007, 12, 25),  date(2008, 1, 1), date(2008, 1, 21), date(2008, 2, 18), date(2008, 5, 22), date(2008, 5, 26), date(2008, 7, 4),  date(2008, 9, 1), date(2008, 10, 13), date(2008, 11, 11), date(2008, 11, 27), date(2008, 12, 25)  ]
def days_from_nearest_holiday(year, month, day): 
    d = date(year, month, day) 
    x = [(abs(d-h)).days for h in holidays]
    return min(x)

## Perform analysis flight data

In [56]:
# Load data and get the shape of the dataframe
# cols = ['year', 'month', 'day', 'dow', 'DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime', 'Carrier', 'FlightNum', 
#         'TailNum', 'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay', 'DepDelay', 'Origin', 'Dest', 
#         'Distance', 'TaxiIn', 'TaxiOut', 'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay', 
#         'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'];
columns = ['year', 'month', 'day_of_month', 'day_of_week', 'dep_time', 'crs_dep_time', 'arr_time', 'crs_arr_time', 'unique_carrier', 'flight_num', 'tail_num',
           'actual_elapsed_time', 'crs_elapsed_time', 'air_time', 'arr_delay', 'dep_delay', 'origin', 'dest', 'distance', 'taxi_in', 'taxi_out','cancelled',
           'cancellation_code', 'diverted', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']

PROCESSING_FILE_NAME = '1996.csv'
processing_file_name = f"./result/{PROCESSING_FILE_NAME}"
flight_df = read_csv_from_hdfs(processing_file_name, cols)

flight_df.shape

(5034804, 29)

In [47]:
flight_df.head()

,year,month,day,dow,DepTime,CRSDepTime,ArrTime,CRSArrTime,Carrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,1996,1,29,1,1996-01-29 20:39:00,1996-01-29 19:30:00,1996-01-29 22:45:00,1996-01-29 21:39:00,DL,345,...,6,10,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1996,1,30,2,1996-01-30 19:31:00,1996-01-30 19:30:00,1996-01-30 21:42:00,1996-01-30 21:39:00,DL,345,...,5,22,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,1996,1,31,3,1996-01-31 19:56:00,1996-01-31 19:30:00,1996-01-31 22:31:00,1996-01-31 21:39:00,DL,345,...,7,27,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,1996,1,1,1,1996-01-01 17:30:00,1996-01-01 15:50:00,1996-01-01 19:09:00,1996-01-01 17:45:00,DL,411,...,4,14,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,1996,1,2,2,1996-01-02 17:14:00,1996-01-02 15:50:00,1996-01-02 18:41:00,1996-01-02 17:45:00,DL,411,...,4,8,0,0,NaN,NaN,NaN,NaN,NaN,NaN


## Let see some flight statistics
Let's look at some basic statistics, after limiting ourselves to flights originating from ORD

In [57]:
# Convert dataframe to correct type
flight_df.columns

def convert_df_types(dataframe):
    dataframe['cancelled'] = dataframe['Cancelled'].astype(bool)
    dataframe['Diverted'] = dataframe['Diverted'].astype(bool)
    dataframe['DepDelay'] = dataframe['DepDelay'].astype(int)
#     dataframe["DepDelay"] = pd.to_numeric(dataframe["DepDelay"], downcast='float')
    
convert_df_types(flight_df)
# flight_df.head()
flight_df['DepDelay']

0           69
1            1
2           26
3          100
4           84
          ... 
5034799      1
5034800      9
5034801     -1
5034802     59
5034803     82
Name: DepDelay, Length: 5034804, dtype: int64

In [58]:
# 
df = flight_df[flight_df['Origin']=='ORD'].dropna(subset=['DepDelay'])
df['DepDelayed'] = df['DepDelay'].apply(lambda x: x>=15)
print("total flights: " + str(df.shape[0]))
print("total delays: " + str(df['DepDelayed'].sum()))

total flights: 264436
total delays: 57183


## Predict

In [5]:
class Configuration:
    PG_HOST = 'business-intelligent-db.cw1neqwhyrda.eu-central-1.rds.amazonaws.com'
    PG_USER = 'postgres'
    PG_PWD = '52eXzbKbqmY2x45f'

from psycopg2 import connect
import pandas.io.sql as psql

# connect to our postgres DB
conn = connect(host=Configuration.PG_HOST,
                user=Configuration.PG_USER,
                password=Configuration.PG_PWD,
                database="airline",
                port=5432)

df = psql.read_sql('''
                    select dep_delay, month, day_of_month, day_of_week, crs_dep_time::time, distance, dest
                    from flight
                    where dep_delay > 15
                    and year = 1995
                ''', conn)


In [39]:
cols = ['dep_delay', 'month', 'day_of_month', 'day_of_week', 'dest',  'distance']
df_1996 = df_1996[cols]

In [2]:
# read files
cols = ['dep_delay', 'month', 'day_of_month', 'day_of_week', 'dest',  'distance']
col_types = {'dep_delay': int, 'month': int, 'day_of_month': int, 'day_of_week': int, 'distance': int, 
            'dest': str}
df_1995 = pd.read_csv('result/1995.csv')

# df_1996 = pd.read_csv('result/1996.csv')
# df_1996 = df_1996[cols]



In [19]:
df_1995['flight_num'].head()

0    482
1    482
2    482
3    482
4    482
Name: flight_num, dtype: int64

In [3]:
cols = ['unique_carrier', 'month', 'flight_num', 'day_of_month', 'day_of_week', 'dest', 'origin', 'dep_time', 'distance', 'dep_delay']
df_1995_selc = df_1995[cols]

In [4]:
df_1995_selc['dep_delay'] = (df_1995_selc['dep_delay'] > 10) *1
categ_cols = ['unique_carrier', 'flight_num', 'dest', 'origin']

for item  in categ_cols:
    df_1995_selc[item] = df_1995_selc[item].astype('category').cat.codes + 1


In [5]:
train, test, y_train, y_test = train_test_split(df_1995_selc.drop(['dep_delay'], axis=1), df_1995_selc["dep_delay"],
                                                random_state=10, test_size=0.4)



In [6]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
enc.fit(train["unique_carrier"].values.reshape(-1, 1))  

# enc.feature_indices_
airline_onehot = enc.transform(train["unique_carrier"].values.reshape(-1, 1)).toarray()
airline_onehot_test = enc.transform(test["unique_carrier"].values.reshape(-1, 1)).toarray()

In [7]:

air_oh_df = pd.DataFrame(airline_onehot, columns = ["A1","A2","A3","A4","A5","A6","A7",
                                                    "A8","A9","A10"])
air_oh_test_df = pd.DataFrame(airline_onehot_test, columns = ["A1","A2","A3","A4","A5","A6","A7",
                                                         "A8","A9","A10"])

In [8]:
train2 = pd.concat([train.reset_index(),air_oh_df.reset_index()],axis=1).drop(["index","month","day_of_month","day_of_week","unique_carrier","flight_num","dest","origin"],axis=1)
test2  = pd.concat([test.reset_index(),air_oh_test_df.reset_index()],axis=1).drop(["index","month","day_of_month","day_of_week","unique_carrier","flight_num","dest","origin"],axis=1)

test2

,dep_time,distance,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10
0,1855,480.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,834,251.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,700,1068.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,827,2182.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,835,1262.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2087651,1537,819.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2087652,834,512.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2087653,1617,612.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2087654,1535,92.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# showing the performance of a classification model for binary
def auc(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))
# Parameter Tuning
param_dist = {"max_depth": [10,30,50],
              "min_child_weight" : [1,3,6],
             "n_estimators": [200],
              "learning_rate": [0.05,0.1,0.16]}

In [10]:
clf_rf = RandomForestClassifier(n_estimators=50, n_jobs=-1)
clf_rf.fit(train2, y_train)

RandomForestClassifier(n_estimators=50, n_jobs=-1)

In [13]:
# Evaluate on test set
pr = clf_rf.predict(test2)

# print results
cm = confusion_matrix(y_test, pr)
print("Confusion matrix")
print(pd.DataFrame(cm))
report_svm = precision_recall_fscore_support(list(y_test), list(pr), average='micro')
print("\nprecision = %0.2f, recall = %0.2f, F1 = %0.2f, accuracy = %0.2f\n" % \
        (report_svm[0], report_svm[1], report_svm[2], accuracy_score(list(y_test), list(pr))))

Confusion matrix
         0       1
0  1625365   51372
1   118565  292354

precision = 0.92, recall = 0.92, F1 = 0.92, accuracy = 0.92



In [14]:
y_test2 = pd.DataFrame({"y":y_test.reset_index(drop=True)})
predict2 = pd.DataFrame({"yhat":pr})
x_test2 = pd.DataFrame(test2)
result = x_test2.join(y_test2)
predicted_result = result.join(predict2)
predicted_result

,dep_time,distance,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,y,yhat
0,1855,480.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0
1,834,251.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0
2,700,1068.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3,827,2182.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,835,1262.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087651,1537,819.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,1
2087652,834,512.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0
2087653,1617,612.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2087654,1535,92.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0


In [16]:
predicted_result[predicted_result['y'] != predicted_result['yhat']]

,dep_time,distance,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,y,yhat
2,700,1068.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
34,1100,303.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0
48,1245,410.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
51,1536,592.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
54,2120,229.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087618,724,2704.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,0
2087624,2208,888.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,0
2087628,1313,883.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0
2087633,1817,408.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,0


In [4]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Create training set and test set
train_y = df_1995['dep_delay'] >= 15
categ = [cols.index(x) for x in [ 'month', 'day_of_month', 'day_of_week', 'dest']]
# enc = ColumnTransformer([('one_hot_encoder', OneHotEncoder(categories='auto',handle_unknown = 'ignore'), categ)],
#                         remainder='passthrough')

df = df_1995.drop('dep_delay', axis=1)
df['dest'] = pd.factorize(df['dest'])[0]
train_x = enc.fit_transform(df)



In [5]:
test_y = df_1996['dep_delay'] >= 15
df = df_1996.drop('dep_delay', axis=1)
df['dest'] = pd.factorize(df['dest'])[0]
test_x = enc.transform(df)


In [7]:
# Create Random Forest classifier with 50 trees
clf_rf = RandomForestClassifier(n_estimators=50, n_jobs=-1)


In [8]:
clf_rf.fit(train_x, train_y)

KeyboardInterrupt: 

In [ ]:


# Evaluate on test set
pr = clf_rf.predict(test_x.toarray())

# print results
cm = confusion_matrix(test_y, pr)
print("Confusion matrix")
print(pd.DataFrame(cm))
report_svm = precision_recall_fscore_support(list(test_y), list(pr), average='micro')


In [25]:
df['dest']

0            0
1            0
2            0
3            0
4            0
          ... 
5035153    212
5035154    212
5035155    212
5035156    212
5035157    212
Name: dest, Length: 5035158, dtype: int64